In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 뉴스 토픽 분류 (with HuggingFace Transformers)

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

본 노트북에서는 `klue/roberta-base` 모델을 **[데이콘(주) 뉴스 토픽 분류 AI 경진대회](https://dacon.io/competitions/official/235747/data)** 데이터셋을 활용하여 모델을 훈련하겠습니다.

학습을 통해 얻어질 `klue-roberta-base-ynat` 모델은 입력된 기사의 카테고리를 예측합니다.

모든 소스 코드는 [`huggingface-notebooks`](https://github.com/huggingface/notebooks)를 참고하였습니다.

먼저, 노트북을 실행하는데 필요한 라이브러리를 설치합니다. 모델 훈련을 위해서는 `transformers`가, 학습 데이터셋 로드를 위해서는 `datasets` 라이브러리의 설치가 필요합니다. 그 외 모델 성능 검증을 위해 `scipy`, `scikit-learn`을 추가로 설치해주도록 합니다.

In [2]:
!pip install -U transformers==4.7.0 datasets scipy scikit-learn

     |████████████████████████████████| 2.5 MB 15.6 MB/s 
     |████████████████████████████████| 325 kB 70.1 MB/s 
     |████████████████████████████████| 38.1 MB 14.0 MB/s 
     |████████████████████████████████| 3.3 MB 51.9 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 136 kB 15.7 MB/s 
     |████████████████████████████████| 1.1 MB 38.1 MB/s 
     |████████████████████████████████| 212 kB 54.1 MB/s 
     |████████████████████████████████| 312 kB 61.2 MB/s 
     |████████████████████████████████| 311 kB 55.9 MB/s 
     |████████████████████████████████| 312 kB 23.0 MB/s 
     |████████████████████████████████| 311 kB 49.4 MB/s 
     |████████████████████████████████| 311 kB 57.2 MB/s 
     |████████████████████████████████| 306 kB 59.5 MB/s 
     |████████████████████████████████| 298 kB 52.8 MB/s 
     |████████████████████████████████| 298 kB 50.4 MB/s 
     |████████████████████████████████| 290 kB 46.2 MB/s 
     |███████

## 뉴스 토픽 분류 모델 학습

노트북을 실행하는데 필요한 라이브러리들을 임포트합니다.

In [3]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

학습에 필요한 정보를 변수로 기록합니다.

학습 태스크로는 `ynat`를, 배치 사이즈로는 32를 지정하겠습니다.

**[데이콘(주) 뉴스 토픽 분류 AI 경진대회](https://dacon.io/competitions/official/235747/data)** 데이터셋을 활용하였습니다.


In [4]:
model_checkpoint="yobi/klue-roberta-base-ynat"
batch_size = 32
task = "ynat"
num_epochs = 5
learningRate = 2e-5
weight_decay = 0.01
seed_value = 42
num_labels = 7

kaggle에서 경로 에러로 인해 working directory에 파일을 복사하였습니다.

In [5]:
# import shutil

# shutil.copyfile("/kaggle/input/news-topic/train_data.csv", "/kaggle/working/train_data.csv")
# shutil.copyfile("/kaggle/input/news-topic/topic_dict.csv", "/kaggle/working/topic_dict.csv")
# shutil.copyfile("/kaggle/input/news-topic/test_data.csv", "/kaggle/working/test_data.csv")

In [10]:
# %cd '/content/drive/MyDrive/'
# %cd '../input/news-topic/'
%cd '/content/drive/MyDrive/files/news/kor'

/content/drive/MyDrive/files/news/kor


In [11]:
# train = pd.read_csv('train_data.csv')
# test = pd.read_csv('test_data.csv')

train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [12]:
# pandas -> dataset타입으로 변경
dataset_train = Dataset.from_pandas(train)
dataset_validation = Dataset.from_pandas(test)

현재 사용하는 모델에서는 **DatasetDict** 형태를 요구하므로,
**DataFrame** 에서 **Dataset** 변경 했습니다.

In [13]:
#train_test_split()
dataset_train, dataset_validation = dataset_train.train_test_split(test_size=0.2).values()
datasets = dataset_train, dataset_validation
# datasets = dataset_train, dataset_validation = dataset_train.train_test_split(test_size=0.2).values()

In [14]:
import datasets
datasets = datasets.DatasetDict({"train":dataset_train,"test":dataset_validation})

In [15]:
# 데이터셋 확인
datasets

DatasetDict({
    train: Dataset({
        features: ['index', 'title', 'topic_idx'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['index', 'title', 'topic_idx'],
        num_rows: 9131
    })
})

In [16]:
# 데이터셋을 확인해보면
datasets["train"][0]

{'index': 20473, 'title': '女배구 흥국생명도로공사 개막전 테일러 매치 앞두고 신경전', 'topic_idx': 5}

In [17]:
datasets["train"]['title']

['女배구 흥국생명도로공사 개막전 테일러 매치 앞두고 신경전',
 '한투증권 NHN 게임 신작 흥행 실패…목표주가↓',
 '게시판 신세계IC 신세계백화점·구글과 AI서비스 협력',
 '포용적 금융 필요하지만 금융업 기본원리도 준수돼야',
 '터키 에르도안 튀니지 방문…리비아 휴전 논의',
 '장하성 주중대사 中외교부 부부장 만나 한반도 문제 논의',
 '美 경찰 저격 용의자 흑인 개빈 롱…경찰 단독범행종합2보',
 '터키군 러시아에서 S400 미사일 운영훈련 참여',
 '청력 잃는 베토벤 생쥐 첨단 유전자 가위로 치료 성공',
 '초고압·극저온 고체 금속 수소 만들어…81년만에 현실화종합',
 'KB금융 푸르덴셜생명 2.3조원에 인수할 듯…공식발표 예정',
 'IT 강국 맞나 홈페이지 없는 한국 기업들',
 '시위에 놀란 알제리 정부 대학에 조기방학 명령',
 '5G·IoT로 초정밀 골프…SKT브이씨 업무협약',
 '내일날씨 구름 낀 일요일…강원 산간 일부 빙판길 주의',
 '느림보 투수 다나카 20초 룰 큰 문제 없어',
 '필리핀 두테르테 대통령 한국 방문 초청받았다',
 'SK텔레콤 2주 80시간 자율적 선택근무제 도입',
 '이란 G7서 핵합의 유지 대책 협의…이란에 제안종합',
 '그리스 크레타섬서 집중 호우로 4명 사망',
 '조성진·정명훈 티켓 파워 5분만에 1천400석 매진',
 '아우베스 1골1도움 PSG 모나코 꺾고 슈퍼컵 5연패',
 '3차원 지도 역량 강화…네이버 에피폴라 인수종합',
 '시위 장기화에 홍콩 경제 올해 마이너스 성장 우려',
 '北김정은 청와대 타격 전투훈련 참관…참수작전 맞대응종합2보',
 '손보사 13분기 순익 17% 감소…車보험 적자전환',
 '선관위 수상경력 부풀린 배현진에 서면경고',
 '나 때문에 경질 차범근하석주 20년만의 해후',
 '朴대통령 軍장병 철통 대비태세에 北 두려워할 것',
 '통신사 마크 없애고 나만의 폰 꾸미는 커스터마이징 인기',
 '이재경 ㈜두산 부회장 두산건설 회장으로 승진',
 '2020

In [18]:
# 편의를 위해 title 과 label 로 변수를 재배정 하였습니다.
df_train = pd.DataFrame({"title":datasets["train"]['title'], 'label':datasets["train"]['topic_idx']})
df_validation = pd.DataFrame({"title":datasets["test"]['title'], 'label':datasets["test"]['topic_idx']})

In [19]:
# DataFrame -> Dataset타입으로 변경
dataset_train = Dataset.from_pandas(df_train)
dataset_validation = Dataset.from_pandas(df_validation)

데이터셋을 전반적으로 살펴보기 위한 시각화 함수를 아래와 같이 정의합니다.

In [20]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset_train[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

In [21]:
topic = pd.read_csv('topic_dict.csv')
topic

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


앞서 정의한 함수를 활용해 훈련 데이터를 살펴보도록 합시다.

이처럼 데이터를 살펴보는 것의 장점으로는 각 라벨에 어떠한 문장들이 해당하는지에 대한 감을 익힐 수 있다는데에 있습니다.

**[데이콘(주) 뉴스 토픽 분류 AI 경진대회](https://dacon.io/competitions/official/235747/data)**
는 총 7개의 라벨을 지니는 데이터셋임을 확인할 수 있습니다. <br>
**0 IT과학**<br>
**1 경제**<br>
**2 사회** <br>
**3 생활문화** <br>
**4 세계** <br> 
**5 스포츠** <br>
**6 정치** <br> 

시각화을 위한 함수를 실행하여 데이터셋을 전반적으로 살펴 봅니다.

In [22]:
show_random_elements(dataset_train)

,title,label
0,북한 김정은 새해 맞아 인민무력부 축하방문,6
1,런던시내 무장경찰 투입 확대…임박한 테러 징후는 없다,4
2,아시안게임 도전 멈춘 3대3농구 여자대표팀 최초라는 ...,5
3,北 위안부 화해·치유재단 설립 사대매국 행위,6
4,얘기 나누는 홍준표유승민,6
5,트럼프 김정은과의 우정도 대선무기…北 바이든 비난에 웃음,4
6,아스널 중국서 식중독 고생…경기선 뮌헨에 극적 승리,5
7,고정금리용 안심대출 나오나…당국 이자 경감방안 검토종합,1
8,터키 EU 제재에도 동지중해 가스 시추 계속,4
9,신간 한국인의 얼굴·몸·뇌·문화,3


훈련 과정 중 모델의 성능을 파악하기 위한 메트릭을 설정합니다.

`datasets` 라이브러리에는 이미 구현된 메트릭을 사용할 수 있는 `load_metric` 함수가 있습니다.

그 중 **GLUE** 데이터셋에 이미 다양한 메트릭이 구현되어 있으므로, **GLUE** 그 중에서도 **KLUE NLI**와 동일한 `accuracy` 메트릭을 사용하는 `qnli` 태스크의 메트릭을 사용합니다.

GLUE: 벤치마크를 통해 BERT 이해하기 <br>
https://huffon.github.io/2019/11/16/glue/

In [23]:
metric = load_metric("accuracy")
# metric = load_metric("glue", "qnli")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

이제 학습에 활용할 토크나이저를 Load 합니다.

In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

데이터셋에서 각 예제들을 뽑아와 토큰화 할 수 있는 함수를 아래와 같이 정의해줍니다.

해당 함수는 모델을 훈련하기 앞서 데이터셋을 미리 토큰화 시켜놓는 작업을 위한 콜백 함수로 사용되게 됩니다.

인자로 넣어주는 `truncation`는 모델이 입력 받을 수 있는 최대 길이 이상의 토큰 시퀀스가 들어오게 될 경우, 최대 길이 기준으로 시퀀스를 자르라는 의미를 지닙니다.

( \* `return_token_type_ids`는 토크나이저가 `token_type_ids`를 반환하도록 할 것인지를 결정하는 인자입니다. `transformers==4.7.0` 기준으로 `token_type_ids`가 기본적으로 반환되므로 `token_type_ids` 자체를 사용하지 않는 `RoBERTa` 모델을 활용하기 위해 해당 인자를 `False`로 설정해주도록 합니다.)

In [25]:
def preprocess_function(examples):
    return tokenizer(
        examples['title'],
        padding="max_length", 
        max_length = 40,
        truncation=True
    )

이제 정의된 전처리 함수를 활용해 데이터셋을 미리 토큰화시키는 작업을 수행합니다.

`datasets` 라이브러리를 통해 얻어진 `DatasetDict` 객체는 `map()` 함수를 지원하므로, 정의된 전처리 함수를 데이터셋 토큰화를 위한 콜백 함수로 `map()` 함수 인자로 넘겨주면 됩니다.

보다 자세한 내용은 [문서](https://huggingface.co/docs/datasets/processing.html#processing-data-with-map)를 참조해주시면 됩니다.

In [26]:
encoded_dataset_train = dataset_train.map(preprocess_function, batched=True)
encoded_dataset_validation = dataset_validation.map(preprocess_function, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

학습을 위한 모델을 로드할 차례입니다.

앞서 살펴본 바와 같이 **Datasets**에는 총 7개의 클래스가 존재하므로, 7개의 클래스를 예측할 수 있는 *SequenceClassification* 구조로 모델을 로드하도록 합니다.

In [27]:
encoded_dataset_train[0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [2,
  1976,
  10056,
  23411,
  6896,
  5634,
  13639,
  19438,
  7752,
  5018,
  2088,
  16425,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'label': 5,
 'title': '女배구 흥국생명도로공사 개막전 테일러 매치 앞두고 신경전',
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

마지막으로 앞서 정의한 메트릭을 모델 예측 결과에 적용하기 위한 함수를 정의합니다.

입력으로 들어오는 `eval_pred`는 [*EvalPrediction*](https://huggingface.co/transformers/internal/trainer_utils.html#transformers.EvalPrediction) 객체이며, 모델의 클래스 별 예측 값과 정답 값을 지닙니다.

클래스 별 예측 중 가장 높은 라벨을 `argmax()`를 통해 뽑아낸 후, 정답 라벨과 비교를 하게 됩니다.

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions)
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
encoded_dataset_train[0]['title'], encoded_dataset_train[0]['label']

이제 앞서 정의한 정보들을 바탕으로 `transformers`에서 제공하는 *Trainer* 객체를 활용하기 위한 인자 관리 클래스를 초기화합니다.

`metric_name`은 앞서 얻어진 메트릭 함수를 활용했을 때, 아래와 같이 `dict` 형식으로 결과 값이 반환되는데 여기서 우리가 사용할 *key* 를 정의해준다고 생각하시면 됩니다.

```python
>>> metric.compute(predictions=fake_preds, references=fake_labels)
{'accuracy': 0.515625}
```

각 인자에 대한 자세한 설명은 [문서](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)에서 참조해주시면 됩니다.

In [ ]:
metric_name = "accuracy"


args = TrainingArguments(
    #"test-nli",
    "test-ynat",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    learning_rate= learningRate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=weight_decay,
    #seed = seed_value,
    load_best_model_at_end=True,
    metric_for_best_model= metric_name
)

이제 로드한 모델, 인자 관리 클래스, 데이터셋 등을 *Trainer* 클래스를 초기화에 넘겨주도록 합니다.

(TIP: Q: 이미 `encoded_datasets`을 만드는 과정에 토큰화가 이루어졌는데 토크나이저를 굳이 넘겨주는 이유가 무엇인가요?,<br>A: 토큰화는 이루어졌지만 학습 과정 시, 데이터를 배치 단위로 넘겨주는 과정에서 배치에 포함된 가장 긴 시퀀스 기준으로 `truncation`을 수행하고 최대 길이 시퀀스 보다 짧은 시퀀스들은 그 길이만큼 `padding`을 수행해주기 위함입니다.)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

이제 정의된 *Trainer* 객체를 다음과 같이 훈련시킬 수 있습니다.

에폭이 지남에 따라 *Loss* 는 떨어지고, 앞서 선정한 메트릭인 *Accuracy* 는 증가하는 것을 확인할 수 있습니다.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


*Trainer* 는 학습을 마치게 되면, `load_best_model_at_end=True` 인자에 따라 메트릭 기준 가장 좋은 성능을 보였던 체크포인트를 로드하게 됩니다.

본 노트북에서는 마지막 에폭 때 가장 좋은 성능을 얻었기에 `evaluate`를 수행해도 같은 결과가 나오겠습니다.

정확도가 약 95% 나왔습니다.

In [ ]:
trainer.evaluate()

지금까지 `transformers`를 라이브러리 내 문장 분류 모델을 학습하는 과정을 **[데이콘(주) 뉴스 토픽 분류 AI 경진대회](https://dacon.io/competitions/official/235747/data)** 데이터셋을 통해 알아보았습니다.

APPENDIX: 앞서 학습된 모델을 HuggingFace 모델 허브에 업로드된, `pipeline` 함수를 통해 사용이 가능합니다.

먼저 `text-classification` 태스크로 파이프라인 객체를 초기화합니다.

( \* `return_all_scores`는 모델이 입력 문장에 대해 측정한 각 라벨에 대한 확률 값을 모두 보여줄 것인지를 결정하는 인자입니다.)

In [ ]:
from transformers import pipeline

classifier = pipeline(
    task = "text-classification",
    model="yobi/klue-roberta-base-ynat",
    return_all_scores=True,
)

In [ ]:
dataset_train[0]['title']

In [ ]:
classifier(dataset_train[0]['title'])

In [ ]:
print(dataset_train[0]['label'])

In [ ]:
classifier(dataset_validation[0]['title'])

In [ ]:
datasets['test']['title'][0]

In [ ]:
print(dataset_validation[0]['label'])

~~***손은 눈보다 빠르다!***~~